In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plotly visualization library
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly_express as px

## Reading files and preprocessing

In [81]:
input_path = '../data/MovementAAL/dataset/MovementAAL_RSS_'
target_path = '../data/MovementAAL/dataset/MovementAAL_target.csv'
group_path = '../data/MovementAAL/groups/MovementAAL_DatasetGroup.csv'

### -------------------------------------------------------- loading files
file_dfs = []
file_lens = []

for i in range(1,315):
    file_path = input_path + str(i) + '.csv'
    file_df = pd.read_csv(file_path, header=0).reset_index(drop=True)
    file_df['_id'] = i
    file_df = file_df.reset_index(drop=True)
    
   
    file_dfs.append(file_df)
    file_lens.append(file_df.shape[0])
    

input_df = pd.concat(file_dfs)
target_df = pd.read_csv(target_path)[' class_label']
group_df = pd.read_csv(group_path)[' dataset_ID']

gr_li = []
for idx, (id_num, target, group) in enumerate(zip(input_df['_id'].unique(), target_df, group_df)):
    gr = input_df[input_df['_id'] == id_num]
    gr = gr.reset_index(drop=True)
    gr['target'] = [target]*gr.shape[0]
    gr['group'] = [group]*gr.shape[0]
    gr['time'] = np.arange(0, gr.shape[0]/8, 1/8)
    gr_li.append(gr)


df = pd.concat(gr_li)
df = df.reset_index(drop=True)
df.columns = ['RSS_anchor1', 'RSS_anchor2', 'RSS_anchor3', 'RSS_anchor4', '_id',
        'target','group', 'time']
df = df[['_id', 'time', 'RSS_anchor1', 'RSS_anchor2', 'RSS_anchor3', 'RSS_anchor4',
        'target', 'group']]
df['target_label'] = df['target'].apply(lambda x: 'Movement' if x==1 else 'Non-Movement')
df.to_csv('../data/indoor_movement.csv', index = False)

## Visualization

In [137]:
df_red = df[df['_id'].isin(np.arange(25, 30))]
df_red.to_csv('../data/indoor_movement_red.csv', index = False)

# signal visulalization

i=1
fig = px.line(df_red, x="time", y="RSS_anchor{}".format(i), color="target_label", line_group="_id", 
              hover_name="target", line_shape="linear", width = 950, height = 550, line_dash = '_id',
              facet_row = 'target_label', color_discrete_sequence = px.colors.qualitative.Alphabet)
iplot(fig)


In [144]:
px.scatter(df_red, x="RSS_anchor2", y="RSS_anchor3", color="target_label", 
              hover_name="target", width = 950, height = 550, marginal_x = 'violin' , 
           marginal_y = 'histogram', color_discrete_sequence = px.colors.qualitative.Alphabet[-7:-5]
              )

In [147]:
! conda install -c conda-forge umap-learn -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/redowan/anaconda3

  added / updated specs: 
    - umap-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.14               |           py37_0         2.1 MB  conda-forge
    umap-learn-0.3.8           |           py37_0          61 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.1 MB

The following NEW packages will be INSTALLED:

    umap-learn: 0.3.8-py37_0  conda-forge

The following packages will be UPDATED:

    conda:      4.5.12-py37_0             --> 4.6.14-py37_0 conda-forge


conda-4.6.14         | 2.1 MB    | #####